In [1]:
import requests
import pandas as pd 

# Define the base URL and the API endpoint
BASE_URL = "https://opendata.wuerzburg.de/"  
ENDPOINT = "api/explore/v2.1/catalog/datasets/passantenzaehlung_{dataset_type}/exports/json"

# Constants for dataset types
DATASET_TAGES = "tagesdaten"
DATASET_STUNDEN = "stundendaten"


def get_data(dataset_type: str):

    if dataset_type not in [DATASET_TAGES, DATASET_STUNDEN]:
        print(f"Invalid dataset type: {dataset_type}")
        return

    url = BASE_URL + ENDPOINT.format(dataset_type=dataset_type)

    try:
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()  # Parse the JSON response
            print("API call successful!")
            print(f"{len(data)} records received.")
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Error occurred: {e}")

    columns = ['timestamp', 'temperature', 'pedestrians_count', 'location_name']

    df = pd.DataFrame.from_records(data=data, columns=columns)

    # Rename location_name to location 
    df.rename(columns={'location_name': 'location'}, inplace=True)

    # Convert the 'timestamp' column to datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Convert to CET (Central European Time) timezone if the dataset type is DATASET_STUNDEN
    if dataset_type == DATASET_STUNDEN:
        df['timestamp'] = df['timestamp'].dt.tz_convert('CET')

    return df



In [2]:
TAGES_ODER_STUNDEN = DATASET_STUNDEN

In [3]:
df = get_data(TAGES_ODER_STUNDEN)
df

API call successful!
20259 records received.


,timestamp,temperature,pedestrians_count,location
0,2024-09-23 23:00:00+02:00,15.3,0,Schönbornstraße
1,2024-09-23 23:00:00+02:00,15.3,0,Spiegelstraße
2,2024-09-23 23:00:00+02:00,15.3,3,Kaiserstraße
3,2024-09-24 00:00:00+02:00,15.0,29,Schönbornstraße
4,2024-09-24 00:00:00+02:00,15.0,33,Spiegelstraße
...,...,...,...,...
20254,2024-10-12 11:00:00+02:00,7.0,1215,Spiegelstraße
20255,2024-10-12 11:00:00+02:00,7.0,1859,Kaiserstraße
20256,2024-10-12 12:00:00+02:00,9.0,4283,Schönbornstraße
20257,2024-10-12 12:00:00+02:00,9.0,1453,Spiegelstraße


In [4]:
df.shape

(20259, 4)

In [5]:
def add_missing_rows(df, dataset_type: str):

    # Add missing rows for each location and timestamp

    if dataset_type not in [DATASET_TAGES, DATASET_STUNDEN]:
        print(f"Invalid dataset type: {dataset_type}")
        return

    # Get the unique locations 
    locations = df['location'].unique()

    # Define new empty dataframe with the same columns as the original dataframe
    df_new = pd.DataFrame(columns=df.columns)

    for location in locations:
        mask = df['location'] == location
        df_location = df[mask]

        # Get the min and max timestamp for the location
        timestamp_min = df_location['timestamp'].min()
        timestamp_max = df_location['timestamp'].max()

        # Create a full range of timestamps for the location
        full_range = pd.date_range(start=timestamp_min, end=timestamp_max, freq='h' if dataset_type == DATASET_STUNDEN else 'D')

        # Merge the full range with the location data
        df_location = pd.DataFrame({'timestamp': full_range}).merge(df_location, on='timestamp', how='left')


        # Fill NaN values with the previous value
        df_location.ffill(inplace=True)

        # Fill remaining NaN values with the next value
        df_location.bfill(inplace=True)

        # Append the location data to the new dataframe
        df_new = pd.concat([df_new, df_location])


    return df_new


In [6]:
df= add_missing_rows(df, TAGES_ODER_STUNDEN)
df

/var/folders/4j/tc3nzkz10kd6976_d_mmfh680000gp/T/ipykernel_78562/2675030903.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, df_location])


,timestamp,temperature,pedestrians_count,location
0,2024-01-01 00:00:00+01:00,5.0,1346.0,Schönbornstraße
1,2024-01-01 01:00:00+01:00,5.0,772.0,Schönbornstraße
2,2024-01-01 02:00:00+01:00,5.0,211.0,Schönbornstraße
3,2024-01-01 03:00:00+01:00,6.0,173.0,Schönbornstraße
4,2024-01-01 04:00:00+01:00,5.0,174.0,Schönbornstraße
...,...,...,...,...
6847,2024-10-12 08:00:00+02:00,3.0,408.0,Kaiserstraße
6848,2024-10-12 09:00:00+02:00,4.0,748.0,Kaiserstraße
6849,2024-10-12 10:00:00+02:00,5.0,1286.0,Kaiserstraße
6850,2024-10-12 11:00:00+02:00,7.0,1859.0,Kaiserstraße


In [7]:
def create_features(df, dataset_type: str):
    
    if dataset_type not in [DATASET_TAGES, DATASET_STUNDEN]:
        print(f"Invalid dataset type: {dataset_type}")
        return

    
    # Add a column 'special_event' True if the day of the timestamp is a special event, False otherwise
    special_events = [
        # Fasching
        '2020-02-23', '2024-02-03', '2024-02-11', '2022-02-13', '2023-02-19',
        
        # Verkaufsoffener Sonntag (Mandelsonntag)
        '2022-10-30', '2023-10-29', '2021-10-31',
        
        # Christmas Market
        '2023-12-01', '2023-12-23', '2022-11-05', '2022-12-23', '2020-11-27', '2020-12-15',
        
        # Stadtfest
        '2022-09-16', '2022-09-17', '2023-09-13', '2023-09-14',
    
        # Residenzlauf
        '2022-05-01', '2023-04-30', '2024-04-28',
        
        # Mozartfest
        '2020-06-09', '2020-06-27', '2021-05-28', '2021-06-27', '2022-05-20', '2022-06-19', '2023-06-02', '2023-07-02', '2024-05-24', '2024-06-23',
        
        # Residenz Weinfest
        '2024-06-28', '2024-07-07', '2023-06-30', '2023-07-09', '2022-07-10',
        
        # Kiliani
        '2022-07-01', '2022-07-17', '2023-06-30', '2023-07-16', '2024-07-05', '2024-07-21',
        
        # Frühjahrsvolksfest
        '2024-03-09', '2024-03-24', '2023-03-18', '2023-04-02', '2022-03-26', '2022-04-10',
        
        # Africa Festival
        '2024-05-30', '2024-06-02', '2023-05-26', '2023-05-29', '2022-05-26', '2022-05-29',
        
        # Street Food Festival
        '2022-05-28', '2022-05-29',
        
        # Allerheiligenmesse
        '2023-10-28', '2023-11-12', '2022-10-29', '2022-11-13',
        
        # Mandelsonntag
        '2021-10-31', '2020-10-31', '2022-10-27', '2023-10-29',
        
        # Hafensommer
        '2024-07-19', '2024-08-04', '2023-07-21', '2023-08-06', '2022-07-22', '2022-08-07', '2021-08-07', '2021-08-08',
        
        # Umsonst & Draußen
        '2024-06-20', '2024-06-23', '2023-06-08', '2023-06-11', '2022-06-16', '2022-06-19', '2021-09-02', '2021-09-05',
        
        # Weindorf
        '2022-05-25', '2022-06-06', '2023-05-26', '2023-06-04', '2024-05-29', '2024-06-09',
        
        # Weinparade
        '2024-08-22', '2024-09-01', '2023-08-24', '2023-09-03', '2022-08-25', '2022-09-01'
    ]

    df['special_event'] = df['timestamp'].dt.strftime('%Y-%m-%d').isin(special_events)


    # Add a column 'holidays' True if the day of the timestamp is a holiday, False otherwise
    holidays = ['2020-01-01', '2020-01-06', '2020-04-10', '2020-04-13', '2020-05-01', '2020-05-08', '2020-05-21',
                 '2020-06-01', '2020-06-11', '2020-08-15', '2020-10-03', '2020-11-01', '2020-12-25', '2020-12-26',
                   '2021-01-01', '2021-01-06', '2021-04-02', '2021-04-05', '2021-05-01', '2021-05-13', '2021-05-24', 
                   '2021-06-03', '2021-08-15', '2021-10-03', '2021-11-01', '2021-12-25', '2021-12-26', '2022-01-01', 
                   '2022-01-06', '2022-04-15', '2022-04-18', '2022-05-01', '2022-05-26', '2022-06-06', '2022-06-16', 
                   '2022-08-15', '2022-10-03', '2022-11-01', '2022-12-25', '2022-12-26', '2023-01-01', '2023-01-06', 
                   '2023-04-07', '2023-04-10', '2023-05-01', '2023-05-18', '2023-05-29', '2023-06-08', '2023-08-15', '2023-10-03',
                     '2023-11-01', '2023-12-25', '2023-12-26', '2024-01-01', '2024-01-06', '2024-03-29', '2024-04-01', '2024-05-01', 
                     '2024-05-09', '2024-05-20', '2024-05-30', '2024-08-15', '2024-10-03', '2024-11-01', '2024-12-25', '2024-12-26'
    ]

    df['holidays'] = df['timestamp'].dt.strftime('%Y-%m-%d').isin(holidays)

    # Add a column 'weekday' with the weekday of the timestamp
    df['weekday'] = df['timestamp'].dt.day_name()

    # Add a column 'hour' with the hour of the timestamp if dataset_type is 'stundendaten'
    if dataset_type == DATASET_STUNDEN:
        df['hour'] = df['timestamp'].dt.hour

    # Add a column 'month' with the month of the timestamp if dataset_type is 'tagesdaten'
    # Add a column 'season' with the season of the timestamp if dataset_type is 'tagesdaten'

    if dataset_type == DATASET_TAGES:
        df['month'] = df['timestamp'].dt.month

        seasons = {
            1: 'Winter',
            2: 'Winter',
            3: 'Spring',
            4: 'Spring',
            5: 'Spring',
            6: 'Summer',
            7: 'Summer',
            8: 'Summer',
            9: 'Fall',
            10: 'Fall',
            11: 'Fall',
            12: 'Winter'
        }

        df['season'] = df['month'].map(seasons)

    return df

In [8]:
df = create_features(df, TAGES_ODER_STUNDEN)
df

,timestamp,temperature,pedestrians_count,location,special_event,holidays,weekday,hour
0,2024-01-01 00:00:00+01:00,5.0,1346.0,Schönbornstraße,False,True,Monday,0
1,2024-01-01 01:00:00+01:00,5.0,772.0,Schönbornstraße,False,True,Monday,1
2,2024-01-01 02:00:00+01:00,5.0,211.0,Schönbornstraße,False,True,Monday,2
3,2024-01-01 03:00:00+01:00,6.0,173.0,Schönbornstraße,False,True,Monday,3
4,2024-01-01 04:00:00+01:00,5.0,174.0,Schönbornstraße,False,True,Monday,4
...,...,...,...,...,...,...,...,...
6847,2024-10-12 08:00:00+02:00,3.0,408.0,Kaiserstraße,False,False,Saturday,8
6848,2024-10-12 09:00:00+02:00,4.0,748.0,Kaiserstraße,False,False,Saturday,9
6849,2024-10-12 10:00:00+02:00,5.0,1286.0,Kaiserstraße,False,False,Saturday,10
6850,2024-10-12 11:00:00+02:00,7.0,1859.0,Kaiserstraße,False,False,Saturday,11


In [9]:
import numpy as np

# Define the lockdown periods
LOCKDOWN_1_START = pd.to_datetime('2020-03-22')
LOCKDOWN_1_END = pd.to_datetime('2020-05-04')

LOCKDOWN_2_START = pd.to_datetime('2020-12-13')
LOCKDOWN_2_END = pd.to_datetime('2021-05-01')

def drop_lockdown(df):

    # Drop rows that fall within the lockdown periods
    df = df[~((df['timestamp'] >= LOCKDOWN_1_START) & (df['timestamp'] <= LOCKDOWN_1_END))]
    df = df[~((df['timestamp'] >= LOCKDOWN_2_START) & (df['timestamp'] <= LOCKDOWN_2_END))]

    # Reset the index
    df.reset_index(drop=True, inplace=True)


    return df

        

    



In [10]:
if TAGES_ODER_STUNDEN == DATASET_TAGES:
    df = drop_lockdown(df)



In [11]:

import numpy as np

def encode_features(df, dataset_type: str):


    def cyclic_encode(df, column, max_val):
        df[f'{column}_sin'] = np.sin(2 * np.pi * df[column]/max_val)
        df[f'{column}_cos'] = np.cos(2 * np.pi * df[column]/max_val)
        return df


    if dataset_type not in [DATASET_TAGES, DATASET_STUNDEN]:
        print(f"Invalid dataset type: {dataset_type}")
        return

    # One-hot encode the 'weekday' column
    df = pd.get_dummies(df, columns=['weekday'])

    # One-hot encode the 'season' column if dataset_type is 'tagesdaten'
    # One-hot encode and cyclilc encode the 'month' column if dataset_type is 'tagesdaten'
    if dataset_type == DATASET_TAGES:
        df = pd.get_dummies(df, columns=['season'])
        df = cyclic_encode(df, 'month', 12)
        df = pd.get_dummies(df, columns=['month'])

    # One-hot encode and cyclic encode the 'hour' column if dataset_type is 'stundendaten'
    if dataset_type == DATASET_STUNDEN:
        df = cyclic_encode(df, 'hour', 24)
        df = pd.get_dummies(df, columns=['hour'])

    return df



In [12]:
df = encode_features(df, TAGES_ODER_STUNDEN)
df

,timestamp,temperature,pedestrians_count,location,special_event,holidays,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,2024-01-01 00:00:00+01:00,5.0,1346.0,Schönbornstraße,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2024-01-01 01:00:00+01:00,5.0,772.0,Schönbornstraße,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2024-01-01 02:00:00+01:00,5.0,211.0,Schönbornstraße,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2024-01-01 03:00:00+01:00,6.0,173.0,Schönbornstraße,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2024-01-01 04:00:00+01:00,5.0,174.0,Schönbornstraße,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6847,2024-10-12 08:00:00+02:00,3.0,408.0,Kaiserstraße,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
6848,2024-10-12 09:00:00+02:00,4.0,748.0,Kaiserstraße,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
6849,2024-10-12 10:00:00+02:00,5.0,1286.0,Kaiserstraße,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
6850,2024-10-12 11:00:00+02:00,7.0,1859.0,Kaiserstraße,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
def get_lags(df, dataset_type: str):


    if dataset_type not in [DATASET_TAGES, DATASET_STUNDEN]:
        print(f"Invalid dataset type: {dataset_type}")
        return
    
    # Sort dataframe by 'location' and 'timestamp'
    df.sort_values(by=['location', 'timestamp'], inplace=True)

    # Get 7-day lag for the 'pedestrians_count' column

    df['pedestrians_count_7d_lag'] = np.nan

    lag = 7 if dataset_type == DATASET_TAGES else 168

    # Split the dataframe into groups based on the 'location' column and get the 7-day lag for each group
    locations  = df['location'].unique()

    for location in locations:
        mask = df['location'] == location
        df.loc[mask, 'pedestrians_count_7d_lag'] = df.loc[mask, 'pedestrians_count'].shift(lag)

    # Drop rows with NaN values
    df.dropna(inplace=True)

    # Sort dataframe by 'timestamp' and 'location'
    df.sort_values(by=['timestamp', 'location'], inplace=True)

    # Reset the index
    df.reset_index(drop=True, inplace=True)

    return df





In [14]:
df = get_lags(df, TAGES_ODER_STUNDEN)
df

,timestamp,temperature,pedestrians_count,location,special_event,holidays,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,pedestrians_count_7d_lag
0,2024-01-08 00:00:00+01:00,-1.6,47.0,Kaiserstraße,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,1197.0
1,2024-01-08 00:00:00+01:00,-1.6,13.0,Schönbornstraße,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,1346.0
2,2024-01-08 00:00:00+01:00,-1.6,24.0,Spiegelstraße,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,388.0
3,2024-01-08 01:00:00+01:00,-2.0,10.0,Kaiserstraße,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,697.0
4,2024-01-08 01:00:00+01:00,-2.0,16.0,Schönbornstraße,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,772.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,2024-10-12 11:00:00+02:00,7.0,3385.0,Schönbornstraße,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,3689.0
20048,2024-10-12 11:00:00+02:00,7.0,1215.0,Spiegelstraße,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,1222.0
20049,2024-10-12 12:00:00+02:00,9.0,2277.0,Kaiserstraße,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,2644.0
20050,2024-10-12 12:00:00+02:00,9.0,4283.0,Schönbornstraße,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,4613.0


In [15]:
TEST_SPLIT_1_DATE = '2024-09-10'
TEST_SPLIT_2_DATE = '2024-09-14'
TEST_SPLIT_3_DATE = '2024-09-26'
TEST_SPLIT_4_DATE = '2024-09-28'
TEST_SPLIT_5_DATE = '2024-09-30'
TEST_SPLIT_6_DATE = '2024-10-02'
TEST_SPLIT_7_DATE = '2024-10-04'
TEST_SPLIT_8_DATE = '2024-10-06'
TEST_SPLIT_9_DATE = '2024-10-08'

TEST_SPLIT_DATE_LONG_TIME_1 = '2024-09-10'
TEST_SPLIT_DATE_LONG_TIME_2 = '2024-09-26'
TEST_SPLIT_DATE_LONG_TIME_3 = '2024-10-03'


SHORT_TIME = 'short_time'
LONG_TIME = 'long_time'

SHORT_OR_LONG_TIME = LONG_TIME

splits = (
    TEST_SPLIT_1_DATE, TEST_SPLIT_2_DATE, TEST_SPLIT_3_DATE, TEST_SPLIT_4_DATE,
    TEST_SPLIT_5_DATE, TEST_SPLIT_6_DATE, TEST_SPLIT_7_DATE, TEST_SPLIT_8_DATE,
    TEST_SPLIT_9_DATE
) if SHORT_OR_LONG_TIME == SHORT_TIME else (
    TEST_SPLIT_DATE_LONG_TIME_1, TEST_SPLIT_DATE_LONG_TIME_2, TEST_SPLIT_DATE_LONG_TIME_3
)


HORIZON = 2 if SHORT_OR_LONG_TIME == SHORT_TIME else 7


def get_weather_forecast(date:str, dataset_type: str):

    if dataset_type not in [DATASET_TAGES, DATASET_STUNDEN]:
        print(f"Invalid dataset type: {dataset_type}")
        return

    # Get the weather forecast for the given date

    path = f'weather/weather_forecast_{date}.csv'

    forecast_df = pd.read_csv(path).rename(columns={'temperature_2m': 'temperature'})

    if dataset_type == DATASET_TAGES:
        # Calculate the average temperature for each day and save it as a dictionary
        temperature_map = forecast_df.groupby('date')['temperature'].mean().round(1).to_dict()
    elif dataset_type == DATASET_STUNDEN:
        # Save the temperature forecast for each hour as a dictionary
        temperature_map = forecast_df.set_index('time')['temperature'].round(1).to_dict()


    return temperature_map



def encode_splits(df, dataset_type:str, n_days:int, *test_split_dates):

    df = df.copy()

    for i, date in enumerate(test_split_dates, start=1):

        # Convert the date to datetime format (CET timezone adds 0:00:00 to the date)
        date = pd.to_datetime(date)

        if dataset_type == DATASET_STUNDEN:
            date = date.tz_localize('CET')

        # Gets the day + 7 days
        date_plus_7 = date + pd.DateOffset(days=n_days)

        df[f'train_{i}'] = df['timestamp'] < date
        df[f'test_{i}'] = (df['timestamp'] >= date) & (df['timestamp'] < date_plus_7)

        
        # Add the weather forecast for the test split
        df[f'temperature_forecast_{i}'] = np.nan

    
        temperature_map = get_weather_forecast(date.strftime('%Y-%m-%d'), dataset_type)
        
        # Fill the temperature forecast for the test split
        format = '%Y-%m-%d %H:%M:%S' if dataset_type == DATASET_STUNDEN else '%Y-%m-%d'
        mask = df[f'test_{i}']
        df.loc[mask, f'temperature_forecast_{i}'] = df.loc[mask, 'timestamp'].dt.strftime(format).map(temperature_map)

        

    return df




In [16]:
df = encode_splits(df, TAGES_ODER_STUNDEN, HORIZON, *splits) 
df

,timestamp,temperature,pedestrians_count,location,special_event,holidays,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,...,pedestrians_count_7d_lag,train_1,test_1,temperature_forecast_1,train_2,test_2,temperature_forecast_2,train_3,test_3,temperature_forecast_3
0,2024-01-08 00:00:00+01:00,-1.6,47.0,Kaiserstraße,False,False,False,True,False,False,...,1197.0,True,False,NaN,True,False,NaN,True,False,NaN
1,2024-01-08 00:00:00+01:00,-1.6,13.0,Schönbornstraße,False,False,False,True,False,False,...,1346.0,True,False,NaN,True,False,NaN,True,False,NaN
2,2024-01-08 00:00:00+01:00,-1.6,24.0,Spiegelstraße,False,False,False,True,False,False,...,388.0,True,False,NaN,True,False,NaN,True,False,NaN
3,2024-01-08 01:00:00+01:00,-2.0,10.0,Kaiserstraße,False,False,False,True,False,False,...,697.0,True,False,NaN,True,False,NaN,True,False,NaN
4,2024-01-08 01:00:00+01:00,-2.0,16.0,Schönbornstraße,False,False,False,True,False,False,...,772.0,True,False,NaN,True,False,NaN,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,2024-10-12 11:00:00+02:00,7.0,3385.0,Schönbornstraße,False,False,False,False,True,False,...,3689.0,False,False,NaN,False,False,NaN,False,False,NaN
20048,2024-10-12 11:00:00+02:00,7.0,1215.0,Spiegelstraße,False,False,False,False,True,False,...,1222.0,False,False,NaN,False,False,NaN,False,False,NaN
20049,2024-10-12 12:00:00+02:00,9.0,2277.0,Kaiserstraße,False,False,False,False,True,False,...,2644.0,False,False,NaN,False,False,NaN,False,False,NaN
20050,2024-10-12 12:00:00+02:00,9.0,4283.0,Schönbornstraße,False,False,False,False,True,False,...,4613.0,False,False,NaN,False,False,NaN,False,False,NaN


In [17]:
# Get all dates where the test_1 column is True
df[df['test_1']][['timestamp', 'temperature','temperature_forecast_1', 'pedestrians_count', 'location']]

,timestamp,temperature,temperature_forecast_1,pedestrians_count,location
17709,2024-09-10 00:00:00+02:00,15.0,15.8,15.0,Kaiserstraße
17710,2024-09-10 00:00:00+02:00,15.0,15.8,17.0,Schönbornstraße
17711,2024-09-10 00:00:00+02:00,15.0,15.8,30.0,Spiegelstraße
17712,2024-09-10 01:00:00+02:00,15.0,15.9,20.0,Kaiserstraße
17713,2024-09-10 01:00:00+02:00,15.0,15.9,13.0,Schönbornstraße
...,...,...,...,...,...
18208,2024-09-16 22:00:00+02:00,12.8,13.3,113.0,Schönbornstraße
18209,2024-09-16 22:00:00+02:00,12.8,13.3,114.0,Spiegelstraße
18210,2024-09-16 23:00:00+02:00,12.4,13.1,15.0,Kaiserstraße
18211,2024-09-16 23:00:00+02:00,12.4,13.1,3.0,Schönbornstraße


In [18]:
df[df['test_2']][['timestamp', 'temperature','temperature_forecast_2', 'pedestrians_count', 'location']]

,timestamp,temperature,temperature_forecast_2,pedestrians_count,location
18861,2024-09-26 00:00:00+02:00,16.0,14.5,43.0,Kaiserstraße
18862,2024-09-26 00:00:00+02:00,16.0,14.5,43.0,Schönbornstraße
18863,2024-09-26 00:00:00+02:00,16.0,14.5,23.0,Spiegelstraße
18864,2024-09-26 01:00:00+02:00,16.0,14.4,9.0,Kaiserstraße
18865,2024-09-26 01:00:00+02:00,16.0,14.4,14.0,Schönbornstraße
...,...,...,...,...,...
19360,2024-10-02 22:00:00+02:00,9.6,7.7,544.0,Schönbornstraße
19361,2024-10-02 22:00:00+02:00,9.6,7.7,240.0,Spiegelstraße
19362,2024-10-02 23:00:00+02:00,9.4,7.3,27.0,Kaiserstraße
19363,2024-10-02 23:00:00+02:00,9.4,7.3,30.0,Schönbornstraße


In [19]:
# Remove the timezone information from the 'timestamp' column
df['timestamp'] = df['timestamp'].dt.tz_localize(None)
df

,timestamp,temperature,pedestrians_count,location,special_event,holidays,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,...,pedestrians_count_7d_lag,train_1,test_1,temperature_forecast_1,train_2,test_2,temperature_forecast_2,train_3,test_3,temperature_forecast_3
0,2024-01-08 00:00:00,-1.6,47.0,Kaiserstraße,False,False,False,True,False,False,...,1197.0,True,False,NaN,True,False,NaN,True,False,NaN
1,2024-01-08 00:00:00,-1.6,13.0,Schönbornstraße,False,False,False,True,False,False,...,1346.0,True,False,NaN,True,False,NaN,True,False,NaN
2,2024-01-08 00:00:00,-1.6,24.0,Spiegelstraße,False,False,False,True,False,False,...,388.0,True,False,NaN,True,False,NaN,True,False,NaN
3,2024-01-08 01:00:00,-2.0,10.0,Kaiserstraße,False,False,False,True,False,False,...,697.0,True,False,NaN,True,False,NaN,True,False,NaN
4,2024-01-08 01:00:00,-2.0,16.0,Schönbornstraße,False,False,False,True,False,False,...,772.0,True,False,NaN,True,False,NaN,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,2024-10-12 11:00:00,7.0,3385.0,Schönbornstraße,False,False,False,False,True,False,...,3689.0,False,False,NaN,False,False,NaN,False,False,NaN
20048,2024-10-12 11:00:00,7.0,1215.0,Spiegelstraße,False,False,False,False,True,False,...,1222.0,False,False,NaN,False,False,NaN,False,False,NaN
20049,2024-10-12 12:00:00,9.0,2277.0,Kaiserstraße,False,False,False,False,True,False,...,2644.0,False,False,NaN,False,False,NaN,False,False,NaN
20050,2024-10-12 12:00:00,9.0,4283.0,Schönbornstraße,False,False,False,False,True,False,...,4613.0,False,False,NaN,False,False,NaN,False,False,NaN


In [20]:
# Save the dataframe to a CSV file
df.to_csv(f'passantenzaehlung_{TAGES_ODER_STUNDEN}_features_{SHORT_OR_LONG_TIME}.csv', index=False)